In [ ]:
#Initializing libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random as r
import networkx as nx
import scipy.sparse as sp
from collections import defaultdict
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import *
import numpy as np  
import timeit
import math
from matplotlib.path import Path
import matplotlib.patches as patches

In [ ]:
vert = 100
edge = 500
array=np.zeros([vert, vert])
raspr=(edge)/(vert*vert)*2
b=r.randint(1,100)

In [ ]:
#adjacency matrix creation
while edge > 0:
    for i in range(1, vert):
        for j in range(i+1, vert):
            r_raspr = r.random()
            randedge=r.randint(1,100)
            if (r_raspr <= raspr and edge > 0 and array[i,j] == 0):
                array[i,j] = randedge
                array[j,i] = randedge
                edge = edge-1
c=np.count_nonzero(array)
print (edge)
print (c)
print (array)

In [ ]:
#graph parameters initialization
def get_matrix_triad(coo_matrix , data=False):
	if not sp.isspmatrix_coo(coo_matrix):
		coo_matrix = sp.coo_matrix(coo_matrix)
	temp = np.vstack((coo_matrix.row , coo_matrix.col , coo_matrix.data)).transpose()
	return temp.tolist()
edags = get_matrix_triad(array)

In [ ]:
#graph initialization
G = nx.Graph()
H = nx.path_graph(array.shape[0])
G.add_nodes_from(H)
G.add_weighted_edges_from(edags)
colors = np.arange(array.shape[0])
#A color map is created according to the weight of the links. 
#The higher the weight (from 0 to 100, cm adjacency matrix generator), 
#the darker the element in accordance with the specified type of color map plt.cm
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

In [ ]:
tt1 = []
tt2 = []
def timecounter():
    rez1 = 0
    rez2 = 0 
    for i in range(0,10):
        t1 = 0
        t2 = 0
        start1 = timeit.default_timer()
        nx.single_source_dijkstra(G, b)
        end1 = timeit.default_timer()
        start2 = timeit.default_timer()
        nx.single_source_bellman_ford(G, b)
        end2 = timeit.default_timer()
        t1 = end1 - start1
        t2 = end2 - start2
        tt1.append(t1)
        tt2.append(t2)
        rez1 += t1
        rez2 += t2
    rez1 = rez1/10
    rez2 = rez2/10
    return rez1, rez2

In [ ]:
def heuristic(point1, point2):
    h = math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)
    return h

vert_ken = 10
hor_len = 20

M = nx.grid_2d_graph(vert_ken, hor_len)  #grid
M.add_edges_from([
    ((x, y), (x+1, y+1))
    for x in range(9)
    for y in range(19)
    ] + [
    ((x+1, y), (x, y+1))
    for x in range(9)
    for y in range(19)
    ], weight=1.4)
pos = nx.spring_layout(M, iterations=1000, seed=4)
nx.draw(M, pos, node_size=5)
plt.show()

removed = []
while (len(removed) < 40):
    for cell_i in np.random.choice(vert_ken, 1):
        for cell_j in np.random.choice(hor_len, 1):
            # print(cell_i, cell_j)
            point = (cell_i, cell_j)
            if point not in removed:
                removed.append(point)
M.remove_nodes_from(removed)

nx.draw(M, pos, node_size=5)
plt.show()


In [ ]:
#creating table format
class Table(Frame):
    def __init__(self, parent=None, headings=tuple(), rows=tuple()):
        super().__init__(parent)

        table = ttk.Treeview(self, show="headings", selectmode="browse")
        table["columns"]=headings
        table["displaycolumns"]=headings

        for head in headings:
            table.heading(head, text=head, anchor=tk.CENTER)
            table.column(head, anchor=tk.CENTER)

        for row in rows:
            table.insert('', tk.END, values=tuple(row))

        scrolltable = tk.Scrollbar(self, command=table.yview)
        table.configure(yscrollcommand=scrolltable.set)
        scrolltable.pack(side=tk.RIGHT, fill=tk.Y)
        table.pack(expand=tk.YES, fill=tk.BOTH)

In [ ]:
#buttons functions
def button1():
    matplotlib.use('TkAgg')
    plt.figure(figsize=(20,10))
    #about edge_cmap look "Graph initialization field comments"
    fig = nx.draw(G, pos = nx.circular_layout(G), node_color=colors, node_size = 300, edgelist=edges, edge_color=weights, width=1.0, edge_cmap=plt.cm.PuBuGn)
    plt.show()
def button2():
    matplotlib.use('TkAgg')
    
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)

    colorlist = ['b', 'g', 'r', 'y', 'grey']


    removed_lst_1 = []
    removed_lst_2 = []
    for i in removed:
        removed_lst_1.append(i[0]-0.5)
        removed_lst_2.append(i[1]-0.5)

    z=0 
    while z < (len(removed)):
        patch1 = patches.Rectangle((removed_lst_1[z], removed_lst_2[z]), 1, 1, edgecolor = 'black', facecolor = 'black', fill=True)
        ax.add_patch(patch1)
        z+=1

    a_star_rez = []
    i = 0
    while i < 5:
        while 1:
            point1 = (r.randint(0, vert_ken-1), r.randint(0, hor_len-1))
            point2 = (r.randint(0, vert_ken-1), r.randint(0, hor_len-1))
            if (point1 not in removed) and (point2 not in removed):
                break
        a_star_rez.append((point1, point2))
        path1 = nx.astar_path(M, point1, point2, heuristic)

        j=0
        while j < (len(path1)-1):
            verts = [
                    path1[j],
                    path1[j+1]
                    ]
            codes = [Path.MOVETO,
                    Path.LINETO]
            path2 = Path(verts, codes)
            patch = patches.PathPatch(path2, facecolor=colorlist[i], edgecolor=colorlist[i], lw=10)
            ax.add_patch(patch)
            j+=1
        i += 1
    plt.grid()
    ax.set_xticks(np.arange(-0.5, 10.5, 1))
    ax.set_yticks(np.arange(-0.5, 20.5, 1))
    plt.draw()
    plt.show()

    top = Toplevel(root)
    #each table for 1-3 ex has three lines by three methods
    table = Table(top, headings=("Dijkstra's algorithm", 'Belman-Ford algoritm', 'Obstacle cells list (four in a row)', 'A* colors','A* algorithm points'),                    
                    rows=(('Mean: ' + str (timecounter()[0]*1000)[:5] + 'ms', 'Mean: ' + str (timecounter()[1]*1000)[:5] + 'ms', str(removed[0:3])  , 'blue'   , 'from ' + str(a_star_rez[0][0]) + ' to ' + str(a_star_rez[0][1])   ), 
                          (    '1 execution: ' + str(tt1[0]*1000)[:5] + 'ms',     '1 execution: ' + str(tt2[0]*1000)[:5] + 'ms', str(removed[4:7])  , 'green'  , 'from ' + str(a_star_rez[1][0]) + ' to ' + str(a_star_rez[1][1])    ),
                          (    '2 execution: ' + str(tt1[1]*1000)[:5] + 'ms',     '2 execution: ' + str(tt2[1]*1000)[:5] + 'ms', str(removed[8:11]) , 'yellow' , 'from ' + str(a_star_rez[2][0]) + ' to ' + str(a_star_rez[2][1])    ),
                          (    '3 execution: ' + str(tt1[2]*1000)[:5] + 'ms',     '3 execution: ' + str(tt2[2]*1000)[:5] + 'ms', str(removed[12:15]), 'red'    , 'from ' + str(a_star_rez[3][0]) + ' to ' + str(a_star_rez[3][1])    ),
                          (    '4 execution: ' + str(tt1[3]*1000)[:5] + 'ms',     '4 execution: ' + str(tt2[3]*1000)[:5] + 'ms', str(removed[16:19]), 'grey'   , 'from ' + str(a_star_rez[4][0]) + ' to ' + str(a_star_rez[4][1])    ), ('5 execution: ' + str(tt1[4]*1000)[:5] + 'ms','5 execution: ' + str(tt2[4]*1000)[:5] + 'ms',str(removed[20:23])), ('6 execution: ' + str(tt1[5]*1000)[:5] + 'ms','6 execution: ' + str(tt2[5]*1000)[:5] + 'ms',str(removed[24:27])), ('7 execution: ' + str(tt1[6]*1000)[:5] + 'ms','7 execution: ' + str(tt2[6]*1000)[:5] + 'ms',str(removed[28:31])), ('8 execution: ' + str(tt1[7]*1000)[:5] + 'ms','8 execution: ' + str(tt2[7]*1000)[:5] + 'ms',str(removed[32:35])), ('9 execution: ' + str(tt1[8]*1000)[:5] + 'ms','9 execution: ' + str(tt2[8]*1000)[:5] + 'ms',str(removed[36:40])), ('10 execution: ' + str(tt1[9]*1000)[:5] + 'ms','10 execution: ' + str(tt2[9]*1000)[:5] + 'ms')
                          )
                )
    table.pack(expand=tk.YES, fill=tk.BOTH)

    canvas = matplotlib.backends.backend_tkagg.FigureCanvasTkAgg(fig, master=root)
    plot_widget = canvas.get_tk_widget()
    plot_widget.grid(row=0, column=0)
    top.mainloop()


In [10]:
#GUI
root = Tk()
root.title("Choose an excercise")
root.geometry("300x100")

#buttons
btn1 = Button(text="Graph visualisation", background="#555", foreground="#ccc",
            padx="20", pady="8", font="16", command=button1)
btn1.place(relx=.2, rely=.1, anchor="c", height=30, width=130, bordermode=OUTSIDE)
btn1.pack(fill=BOTH)
btn2 = Button(text="Dijkstra's, Bellman-Ford and A* algorithms", background="#555", foreground="#ccc",
            padx="20", pady="8", font="16", command=button2)
btn2.place(relx=.2, rely=.2, anchor="c", height=30, width=130, bordermode=OUTSIDE)
btn2.pack(fill=BOTH)

root.mainloop()